In [1]:
from elasticsearch import Elasticsearch
import twarc
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import json
from pprint import pprint                                                                                                
from random import randrange

In [2]:
es = Elasticsearch("http://localhost:9200")

In [3]:
es.info()

ObjectApiResponse({'name': '8cde569aead9', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'ncFi3QThT9-9Ppe2hkkPgQ', 'version': {'number': '7.17.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'e5acb99f822233d62d6444ce45a4543dc1c8059a', 'build_date': '2022-02-23T22:20:54.153567231Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [4]:
# l = []
# with open('tweets1.csv', 'r') as f:
#     lines = f.readlines()
#     for line in tqdm(lines[:1000]):
#         line = int(line.strip())
#         tw = !twarc2 tweet $line
#         l.append(tw)

In [5]:
# with open('downloaded_tweets.csv', 'w') as f:
#     json.dump(l, f)

In [6]:
with open('downloaded_tweets.csv') as json_file:
    data = json.load(json_file)

In [7]:
# with open('downloaded_tweets.csv', 'w') as f:
#     json.dump(l, f)
# yyyyMMdd'T'HHmmss.SSSZ

In [8]:
ldr = []
langs=[]
for i in range(len(data)):
    try:
        author=data[i]['data'][0]['author_id']
        tw_id=data[i]['data'][0]['id']
        tw_text=data[i]['data'][0]['text']
        lang=data[i]['data'][0]['lang']
        date=data[i]['data'][0]['created_at']
        public_metrics=data[i]['data'][0]['public_metrics']
        ldr.append({"author_id": int(author), "tw_id": tw_id, "created_at": date, "public_metrics": public_metrics,
                    "cluster_id": randrange(10), "mlt_message": tw_text, "message": {lang: tw_text}})
        langs.append(lang)
    except KeyError:
        pass

In [9]:
for i in ldr:
    try:
        if list(i['message'].keys()) == ['de']:
            print(i)
    except TypeError:
        pass

{'author_id': 355038897, 'tw_id': '1507146195281616904', 'created_at': '2022-03-25T00:03:49.000Z', 'public_metrics': {'retweet_count': 1343, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'cluster_id': 4, 'mlt_message': 'RT @SGaykwon: Average western Ukraine militia volunteer https://t.co/eCcBi32NkM', 'message': {'de': 'RT @SGaykwon: Average western Ukraine militia volunteer https://t.co/eCcBi32NkM'}}
{'author_id': 1421899973956878341, 'tw_id': '1507146198548946945', 'created_at': '2022-03-25T00:03:49.000Z', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'cluster_id': 3, 'mlt_message': '@Daity1 Wie Putin da wohl drauf reagiert ...', 'message': {'de': '@Daity1 Wie Putin da wohl drauf reagiert ...'}}
{'author_id': 1198369136746860544, 'tw_id': '1507146200671158275', 'created_at': '2022-03-25T00:03:50.000Z', 'public_metrics': {'retweet_count': 92, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'cluster_id': 1, 'mlt_message': 'RT 

In [10]:
try:
    es.indices.delete(index="ukrain_tweets")
    es.indices.create(index="ukrain_tweets")
except:
    es.indices.create(index="ukrain_tweets")

In [11]:
mapit = {"properties":{"author_id":{"type":"keyword"},
                       "tw_id":{"type":"keyword"},
                       "created_at":{"type":"date", "format": "yyyy-MM-dd'T'HH:mm:ss.SSSX"},
                       "public_metrics":{
                           "properties":{
                               "retweet_count":
                                   {"type": "integer"},
                               "reply_count":
                                   {"type": "integer"},
                               "like_count":
                                   {"type": "integer"},
                               "quote_count":
                                   {"type": "integer"},
                           }
                       },
                       "cluster_id":{"type": "integer"},
                       "mlt_message":{"type": "text"},
                       "message":{
                           "properties":{
                                "de": 
                                   {"type":"text","analyzer" : "german"},
                                "en": 
                                   {"type":"text","analyzer" : "english"},
                                "ru": 
                                   {"type":"text","analyzer" : "russian"},
                                "et": 
                                   {"type":"text","analyzer" : "estonian"},
                                "es": 
                                   {"type":"text","analyzer" : "spanish"},
                                "tl": 
                                   {"type":"text"},
                                "pt": 
                                   {"type":"text","analyzer" : "portuguese"},
                                "sv": 
                                   {"type":"text","analyzer" : "swedish"},
                                "it": 
                                   {"type":"text","analyzer" : "italian"},
                                "uk": 
                                   {"type":"text"},
                                "nl": 
                                   {"type":"text","analyzer" : "dutch"},
                                "ja": 
                                   {"type":"text"},                               
                                "tr": 
                                   {"type":"text","analyzer" : "turkish"},
                                "ro": 
                                   {"type":"text","analyzer" : "romanian"},
                                "fr": 
                                   {"type":"text","analyzer" : "french"},
                                "und": 
                                   {"type":"text"},
                                "pl": 
                                   {"type":"text"},
                                "zh": 
                                   {"type":"text"},
                                "cy": 
                                   {"type":"text"},
                                "th": 
                                   {"type":"text", "analyzer" : "thai"},
                                "no": 
                                   {"type":"text", "analyzer": "norwegian"}, 
                                "in": 
                                   {"type":"text"},
                                "ca": 
                                   {"type":"text", "analyzer": "catalan"},
                                "ar": 
                                   {"type":"text", "analyzer": "armenian"}
                           },
                        }
                      }
        }

In [12]:
es.indices.put_mapping(index="ukrain_tweets", body=mapit)

ObjectApiResponse({'acknowledged': True})

In [13]:
es.indices.get_alias(index="*")

ObjectApiResponse({'ukrain_tweets': {'aliases': {}}})

#### Create

In [14]:
for tw in ldr:
    es.index(index="ukrain_tweets", document=tw)

#### Read

all

In [42]:
%%time
res = es.search(index="ukrain_tweets", query={"match_all": {}}, size=100)
res["hits"]["hits"]

Wall time: 21 ms


[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'l_CZ338BRPyB9DDTFkTT',
  '_score': 1.0,
  '_source': {'author_id': 3065680671,
   'tw_id': '1507146205905797124',
   'created_at': '2022-03-25T00:03:51.000Z',
   'public_metrics': {'retweet_count': 25,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 5,
   'mlt_message': 'RT @mattprescott: @AndrewPRLevi It is striking how similar the KLEPTOCRACY, AUTHORITARIANISM, FASCISM and VICTIMHOOD of the RUSSIAN RULING…',
   'message': {'en': 'RT @mattprescott: @AndrewPRLevi It is striking how similar the KLEPTOCRACY, AUTHORITARIANISM, FASCISM and VICTIMHOOD of the RUSSIAN RULING…'}}},
 {'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'mPCZ338BRPyB9DDTFkTp',
  '_score': 1.0,
  '_source': {'author_id': 1283041817659940865,
   'tw_id': '1507146205977100291',
   'created_at': '2022-03-25T00:03:51.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_coun

author id

In [16]:
res = es.search(index="ukrain_tweets", query={"match": {"author_id": "273107148"}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'G_CZ338BRPyB9DDTBESg',
  '_score': 6.469767,
  '_source': {'author_id': 273107148,
   'tw_id': '1507146195101233163',
   'created_at': '2022-03-25T00:03:48.000Z',
   'public_metrics': {'retweet_count': 29,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 8,
   'mlt_message': 'RT @gtlem: UK PM Boris Johnson SNUBBED as Macron and other EU leaders completely ignore him at Nato meeting https://t.co/SsVhz4ZjHP\n\nBut le…',
   'message': {'en': 'RT @gtlem: UK PM Boris Johnson SNUBBED as Macron and other EU leaders completely ignore him at Nato meeting https://t.co/SsVhz4ZjHP\n\nBut le…'}}}]

Checking German & Russian morphology analyzer

In [17]:
res = es.search(index="ukrain_tweets", query={"match": {"message.de": "vielen"}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'KfCZ338BRPyB9DDTdUdR',
  '_score': 2.0509672,
  '_source': {'author_id': 1457699018948190209,
   'tw_id': '1507146268472188930',
   'created_at': '2022-03-25T00:04:06.000Z',
   'public_metrics': {'retweet_count': 21,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 0,
   'mlt_message': 'RT @lisakatharina: Es geht nicht darum, Sanktionen zu unterlaufen. Das müsste man gar nicht. Viele russische Banken sind noch unsanktionier…',
   'message': {'de': 'RT @lisakatharina: Es geht nicht darum, Sanktionen zu unterlaufen. Das müsste man gar nicht. Viele russische Banken sind noch unsanktionier…'}}}]

In [18]:
res = es.search(index="ukrain_tweets", query={"match": {"message.ru": "яйцо"}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'YvCZ338BRPyB9DDTD0Qk',
  '_score': 1.2781154,
  '_source': {'author_id': 549292289,
   'tw_id': '1507146200797097998',
   'created_at': '2022-03-25T00:03:50.000Z',
   'public_metrics': {'retweet_count': 1185,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 0,
   'mlt_message': 'RT @aleks_roza: 300 запорожцев\n\nТаблетка для памяти\n\nОни стояли. \nИх забрасывали яйцами, пакетами с мукой и молоком, камнями. От них требов…',
   'message': {'ru': 'RT @aleks_roza: 300 запорожцев\n\nТаблетка для памяти\n\nОни стояли. \nИх забрасывали яйцами, пакетами с мукой и молоком, камнями. От них требов…'}}}]

Two language search

Решение мультиязычности<br>
Второе решение - сделать вирт. переменную со всеми языками

In [43]:
res = es.search(index="ukrain_tweets", query={"multi_match": {"query": "moment", "fields": ["message.en", "message.de"]}})
res["hits"]["hits"], len(res["hits"]["hits"])

([{'_index': 'ukrain_tweets',
   '_type': '_doc',
   '_id': '0fCZ338BRPyB9DDTaEaf',
   '_score': 5.91999,
   '_source': {'author_id': 2982013384,
    'tw_id': '1507146260356247557',
    'created_at': '2022-03-25T00:04:04.000Z',
    'public_metrics': {'retweet_count': 39,
     'reply_count': 0,
     'like_count': 0,
     'quote_count': 0},
    'cluster_id': 7,
    'mlt_message': 'RT @NinaByzantina: If I had to pick only one token moment of humiliation that Russia, as a civilization, experienced in the late 1980s-1990…',
    'message': {'en': 'RT @NinaByzantina: If I had to pick only one token moment of humiliation that Russia, as a civilization, experienced in the late 1980s-1990…'}}},
  {'_index': 'ukrain_tweets',
   '_type': '_doc',
   '_id': 'qvCZ338BRPyB9DDTYkb_',
   '_score': 4.186263,
   '_source': {'author_id': 15210284,
    'tw_id': '1507146255461494787',
    'created_at': '2022-03-25T00:04:03.000Z',
    'public_metrics': {'retweet_count': 1,
     'reply_count': 1,
     'like_co

Проверю что все работает как надо

In [44]:
res = es.search(index="ukrain_tweets", query={"match": {"message.de": "яйцо"}})
res["hits"]["hits"]

[]

In [46]:
res = es.search(index="ukrain_tweets", query={"match": {"mlt_message": 
                                       {"query":"Putin", "operator": "and"}}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'ZPCZ338BRPyB9DDTWEbr',
  '_score': 3.3071997,
  '_source': {'author_id': 1499763123603050497,
   'tw_id': '1507146250461888519',
   'created_at': '2022-03-25T00:04:02.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 7,
   'mlt_message': 'Özünüzü siklə, Putin!\n(Azerbaijani)\n\n#PutinsWar #StopPutin',
   'message': {'tr': 'Özünüzü siklə, Putin!\n(Azerbaijani)\n\n#PutinsWar #StopPutin'}}},
 {'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'gvCZ338BRPyB9DDTE0TR',
  '_score': 3.2891622,
  '_source': {'author_id': 1497016556563402756,
   'tw_id': '1507146203426959368',
   'created_at': '2022-03-25T00:03:50.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 2,
    'quote_count': 0},
   'cluster_id': 1,
   'mlt_message': "@yagnome_sayin @JustinTrudeau EU, including Mislav have enabled Putin for 20 years. Putin

#### Delete

In [47]:
res = es.search(index="ukrain_tweets", query={"match": {"mlt_message": "яйцами"}})
res["hits"]["hits"]

[]

In [23]:
es.delete(index="ukrain_tweets", id=res["hits"]["hits"][0]['_id'])

ObjectApiResponse({'_index': 'ukrain_tweets', '_type': '_doc', '_id': 'YvCZ338BRPyB9DDTD0Qk', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 967, '_primary_term': 1})

In [24]:
res = es.search(index="ukrain_tweets", query={"match": {"mlt_message": "яйцами"}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'YvCZ338BRPyB9DDTD0Qk',
  '_score': 6.7142386,
  '_source': {'author_id': 549292289,
   'tw_id': '1507146200797097998',
   'created_at': '2022-03-25T00:03:50.000Z',
   'public_metrics': {'retweet_count': 1185,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 0,
   'mlt_message': 'RT @aleks_roza: 300 запорожцев\n\nТаблетка для памяти\n\nОни стояли. \nИх забрасывали яйцами, пакетами с мукой и молоком, камнями. От них требов…',
   'message': {'ru': 'RT @aleks_roza: 300 запорожцев\n\nТаблетка для памяти\n\nОни стояли. \nИх забрасывали яйцами, пакетами с мукой и молоком, камнями. От них требов…'}}}]

#### Update

In [53]:
res = es.search(index="ukrain_tweets", query={"match": {"message.de": "Mensch"}})
res["hits"]["hits"][0]

{'_index': 'ukrain_tweets',
 '_type': '_doc',
 '_id': 'ZvCZ338BRPyB9DDTNEXA',
 '_score': 2.1528335,
 '_source': {'author_id': 887436366530977792,
  'tw_id': '1507146226935996423',
  'created_at': '2022-03-25T00:03:56.000Z',
  'public_metrics': {'retweet_count': 32,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'cluster_id': 8,
  'mlt_message': 'RT @MichaelBonvalot: Lange hat es nicht gedauert: Jetzt beginnen extreme Rechte mit den Attacken auf geflüchtete Menschen aus der Ukraine.…',
  'message': {'de': 'RT @MichaelBonvalot: Lange hat es nicht gedauert: Jetzt beginnen extreme Rechte mit den Attacken auf geflüchtete Menschen aus der Ukraine.…'}}}

In [54]:
new_text = 'Мама мыла раму'
es.update(index='ukrain_tweets', id=res["hits"]["hits"][0]['_id'],
                body={"doc": {'mlt_message': new_text, 'message': {'de': new_text}}})

ObjectApiResponse({'_index': 'ukrain_tweets', '_type': '_doc', '_id': 'ZvCZ338BRPyB9DDTNEXA', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 969, '_primary_term': 1})

In [55]:
res = es.search(index="ukrain_tweets", query={"match": {"_id": res["hits"]["hits"][0]['_id']}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'ZvCZ338BRPyB9DDTNEXA',
  '_score': 1.0,
  '_source': {'author_id': 887436366530977792,
   'tw_id': '1507146226935996423',
   'created_at': '2022-03-25T00:03:56.000Z',
   'public_metrics': {'retweet_count': 32,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 8,
   'mlt_message': 'Мама мыла раму',
   'message': {'de': 'Мама мыла раму'}}}]

### Аггрегации

Выборка по дате

In [28]:
res = es.search(index="ukrain_tweets", query={"range": {"created_at": 
                                       {"gte":"2022-03-25T00:03:50.000Z", "lt": "2022-03-25T00:04:03.000Z"}}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'l_CZ338BRPyB9DDTFkTT',
  '_score': 1.0,
  '_source': {'author_id': 3065680671,
   'tw_id': '1507146205905797124',
   'created_at': '2022-03-25T00:03:51.000Z',
   'public_metrics': {'retweet_count': 25,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 5,
   'mlt_message': 'RT @mattprescott: @AndrewPRLevi It is striking how similar the KLEPTOCRACY, AUTHORITARIANISM, FASCISM and VICTIMHOOD of the RUSSIAN RULING…',
   'message': {'en': 'RT @mattprescott: @AndrewPRLevi It is striking how similar the KLEPTOCRACY, AUTHORITARIANISM, FASCISM and VICTIMHOOD of the RUSSIAN RULING…'}}},
 {'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'mPCZ338BRPyB9DDTFkTp',
  '_score': 1.0,
  '_source': {'author_id': 1283041817659940865,
   'tw_id': '1507146205977100291',
   'created_at': '2022-03-25T00:03:51.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_coun

Популярные твиты

In [29]:
res = es.search(index="ukrain_tweets", query={"range": {"public_metrics.retweet_count": 
                                       {"gte":"30000"}}})
res["hits"]["hits"]

[{'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'HvCZ338BRPyB9DDTT0Yf',
  '_score': 1.0,
  '_source': {'author_id': 1087125851752280064,
   'tw_id': '1507146243264458765',
   'created_at': '2022-03-25T00:04:00.000Z',
   'public_metrics': {'retweet_count': 37439,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'cluster_id': 5,
   'mlt_message': 'RT @richimedhurst: They deleted it but I kept a screenshot.\n\nThe Ukrainian government actually posted this picture, acting like Ukraine has…',
   'message': {'en': 'RT @richimedhurst: They deleted it but I kept a screenshot.\n\nThe Ukrainian government actually posted this picture, acting like Ukraine has…'}}},
 {'_index': 'ukrain_tweets',
  '_type': '_doc',
  '_id': 'dvCZ338BRPyB9DDTNkX-',
  '_score': 1.0,
  '_source': {'author_id': 137356646,
   'tw_id': '1507146228366458880',
   'created_at': '2022-03-25T00:03:56.000Z',
   'public_metrics': {'retweet_count': 30937,
    'reply_count': 0,
    'like_count': 0,
   

Максимальное кол-во ретвитов вообще

In [30]:
res=es.search(index = "ukrain_tweets", query={"match_all": {}}, aggs = {"goose": {"max": {"field": "public_metrics.retweet_count"}}})
print(res['aggregations']['goose'])

{'value': 37440.0}


Максимальное кол-во ретвитов c Путиным

In [31]:
res=es.search(index = "ukrain_tweets", size=1000, 
              query={"multi_match": {"query": "Putin|Путин", "fields": ["message.en", "message.de"]}}, 
                                     aggs = {"goose": {"max": {"field": "public_metrics.retweet_count"}}})
print(res['aggregations']['goose'])

{'value': 20068.0}


Распределение кол-ва ретвитов со словом Путин

In [32]:
res=es.search(index = "ukrain_tweets", size=1000, 
              query={"multi_match": {"query": "Putin|Путин", "fields": ["message.en", "message.de"]}}, 
              aggs = {"goose": {"boxplot": {"field": "public_metrics.retweet_count"}}})
print(len(res['hits']['hits']))
print(res['aggregations']['goose'])

79
{'min': 0.0, 'max': 20068.0, 'q1': 0.0, 'q2': 19.0, 'q3': 802.0, 'lower': 0.0, 'upper': 1914.0}


Среднее кол-во ретвитов со словом Путин

In [33]:
res=es.search(index = "ukrain_tweets", size=1000, query={"match": {"message.en": "Putin"}},
              aggs = {"goose": {"avg": {"field": "public_metrics.retweet_count"}}})
                                                                                                  
print(res['aggregations']['goose']['value'])

1747.9871794871794


Самые популярные кластера со словом Путин

In [34]:
res=es.search(index = "ukrain_tweets", size=1000, query={"match": {"message.en": "Putin"}}, 
              aggs = {"goose": {"terms": {"field": "cluster_id", "order": {"_count": "desc"}}}})

pprint(res['aggregations']['goose'])
# res['hits']['hits']

{'buckets': [{'doc_count': 16, 'key': 9},
             {'doc_count': 11, 'key': 1},
             {'doc_count': 11, 'key': 6},
             {'doc_count': 9, 'key': 8},
             {'doc_count': 8, 'key': 0},
             {'doc_count': 6, 'key': 2},
             {'doc_count': 6, 'key': 3},
             {'doc_count': 5, 'key': 4},
             {'doc_count': 3, 'key': 5},
             {'doc_count': 3, 'key': 7}],
 'doc_count_error_upper_bound': 0,
 'sum_other_doc_count': 0}


Среднее кол-во ретвитов по кластерам по убыванию популярности со словом Путин

In [35]:
req={
    "goose" : {
        "terms": {"field": "cluster_id", "order": {"_count": "desc"}},
        "aggs" : {"goose": {"avg": {"field": "public_metrics.retweet_count"}}}    
            }
    }


res = es.search(index = "ukrain_tweets", size=1000, query={"match": {"mlt_message": "Putin"}}, aggs = req)
#res['aggregations']['goose']['buckets']
# res['hits']['hits']

In [36]:
for i in res['aggregations']['goose']['buckets']:
    print('Cluster:', i['key'], '|', 'Num_of_docs:', i['doc_count'], '|', 'Avr. retweets:', round(i['goose']['value']))

Cluster: 6 | Num_of_docs: 13 | Avr. retweets: 1438
Cluster: 9 | Num_of_docs: 13 | Avr. retweets: 292
Cluster: 8 | Num_of_docs: 12 | Avr. retweets: 648
Cluster: 0 | Num_of_docs: 11 | Avr. retweets: 2248
Cluster: 2 | Num_of_docs: 11 | Avr. retweets: 2112
Cluster: 1 | Num_of_docs: 10 | Avr. retweets: 1380
Cluster: 3 | Num_of_docs: 8 | Avr. retweets: 749
Cluster: 4 | Num_of_docs: 8 | Avr. retweets: 86
Cluster: 5 | Num_of_docs: 5 | Avr. retweets: 2148
Cluster: 7 | Num_of_docs: 5 | Avr. retweets: 123


10/10